In [1]:
# 1. 데이터 다운로드 & 압축 해제
!wget -O data.tar.gz "https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz"
!tar -xzf data.tar.gz

# 2. 파일 확인
!ls -la data/

--2026-01-28 08:59:33--  https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000377/data/data.tar.gz
Resolving aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)... 3.5.185.24, 3.5.185.124, 3.5.185.86, ...
Connecting to aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)|3.5.185.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3280795723 (3.1G) [binary/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   3.05G  36.1MB/s    in 86s     

2026-01-28 09:00:59 (36.4 MB/s) - ‘data.tar.gz’ saved [3280795723/3280795723]

total 16
drwxr-xr-x 4 root root 4096 Dec 17  2024 .
drwxr-xr-x 1 root root 4096 Jan 28 09:00 ..
drwxr-xr-x 4 root root 4096 Jan 27  2024 datasets
drwxr-xr-x 5 root root 4096 Dec 17  2024 pseudo_label


In [3]:
# 필수 라이브러리 재설치
!pip install -q segmentation-models-pytorch albumentations opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.2 MB/s eta 0:00:00


In [4]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm import tqdm
import gc

# 1. 환경 설정 및 A100 최적화
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True

# 2. 경로 설정 (image_dbe68a.png 구조 반영)
BASE_PATH = './data/datasets'
TRAIN_IMG_DIR = os.path.join(BASE_PATH, 'images/train')
VAL_IMG_DIR = os.path.join(BASE_PATH, 'images/val')
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')
TRAIN_JSON = os.path.join(BASE_PATH, 'jsons/train.json')
VAL_JSON = os.path.join(BASE_PATH, 'jsons/val.json')
TEST_JSON = os.path.join(BASE_PATH, 'jsons/test.json')
SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# [고득점을 위한 하이퍼파라미터]
RESIZE_TARGET = 1024       # 작은 글씨 대응 (핵심)
BATCH_SIZE = 4             # 고해상도 고려 A100 40G 기준
ACCUMULATION_STEPS = 8     # 실효 배치 사이즈 32 (4 * 8)
EPOCHS = 30
LEARNING_RATE = 1e-4

# 3. 데이터셋 클래스
class ReceiptDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None, is_test=False):
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        image = cv2.imread(os.path.join(self.img_dir, img_name))
        if image is None: return self.__getitem__((idx + 1) % len(self))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape
        if not self.is_test:
            mask = np.zeros((h, w), dtype=np.float32)
            words = self.data[img_name].get('words', {})
            for word_id in words:
                points = np.array(words[word_id]['points'], dtype=np.int32)
                cv2.fillPoly(mask, [points], 1)
            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                return augmented['image'], augmented['mask']
        else:
            if self.transform:
                augmented = self.transform(image=image)
                return augmented['image'], img_name, (h, w)
        return image

# 4. 강력한 증강 기법 (Unseen Data 대응)
train_transform = A.Compose([
    A.Resize(RESIZE_TARGET, RESIZE_TARGET),
    A.Perspective(scale=(0.05, 0.1), p=0.4), # 촬영 각도 왜곡 대응
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0)),
        A.MotionBlur(blur_limit=3),
    ], p=0.3),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([A.Resize(RESIZE_TARGET, RESIZE_TARGET), A.Normalize(), ToTensorV2()])

# 5. 복합 손실 함수 (BCE + Dice)
class CombinedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.dice = smp.losses.DiceLoss(mode='binary')
        self.bce = nn.BCEWithLogitsLoss()
    def forward(self, y_pred, y_true):
        return 0.5 * self.dice(y_pred, y_true) + 0.5 * self.bce(y_pred, y_true)

# 6. 폴리곤 추출 후처리
def mask_to_polygons(mask, min_area=30):
    polygons = []
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) < min_area: continue
        # 폴리곤 단순화
        epsilon = 0.005 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if len(approx) >= 4:
            polygons.append(approx.reshape(-1, 2).tolist())
    return polygons

def polygons_to_string(polygons):
    if not polygons: return ""
    return "|".join([" ".join([f"{int(p[0])} {int(p[1])}" for p in poly]) for poly in polygons])

# 7. 메인 실행 루프
def main():
    torch.cuda.empty_cache()
    gc.collect()

    train_ds = ReceiptDataset(TRAIN_IMG_DIR, TRAIN_JSON, transform=train_transform)
    val_ds = ReceiptDataset(VAL_IMG_DIR, VAL_JSON, transform=val_transform)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    # 모델 선언 (EfficientNet 백본 강화)
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b3", encoder_weights="imagenet", in_channels=3, classes=1).to(DEVICE)
    model = model.to(memory_format=torch.channels_last)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    criterion = CombinedLoss()
    scaler = GradScaler('cuda')

    # 학습 단계
    print("🚀 고해상도 학습 시작...")
    best_loss = float('inf')
    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_l = 0
        optimizer.zero_grad(set_to_none=True)
        for i, (images, masks) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}")):
            images = images.to(DEVICE, non_blocking=True, memory_format=torch.channels_last)
            masks = masks.to(DEVICE, non_blocking=True).unsqueeze(1)
            with autocast('cuda', dtype=torch.bfloat16):
                loss = criterion(model(images), masks) / ACCUMULATION_STEPS
            scaler.scale(loss).backward()
            if (i + 1) % ACCUMULATION_STEPS == 0:
                scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            train_l += loss.item() * ACCUMULATION_STEPS

        # Validation
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(DEVICE, memory_format=torch.channels_last)
                masks = masks.to(DEVICE).unsqueeze(1)
                v_loss += criterion(model(images), masks).item()

        avg_v = v_loss / len(val_loader)
        print(f"📊 Val Loss: {avg_v:.4f}")
        if avg_v < best_loss:
            best_loss = avg_v
            torch.save(model.state_dict(), 'best_model.pth')
            print("⭐ 모델 저장")
        scheduler.step()

    # 🔍 추론 단계 (TTA 적용)
    print("🔍 TTA 기반 추론 시작...")
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    test_ds = ReceiptDataset(TEST_IMG_DIR, TEST_JSON, transform=val_transform, is_test=True)
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

    sample_df = pd.read_csv(SAMPLE_SUB)
    preds = {}
    with torch.no_grad():
        for img, name, (orig_h, orig_w) in tqdm(test_loader, desc="Inference"):
            img = img.to(DEVICE, memory_format=torch.channels_last)
            with autocast('cuda', dtype=torch.bfloat16):
                # TTA: 원본 + 좌우 반전 예측 평균
                p1 = torch.sigmoid(model(img))
                p2 = torch.sigmoid(model(torch.flip(img, dims=[3])))
                mask = (p1 + torch.flip(p2, dims=[3])) / 2

            mask = cv2.resize(mask.cpu().numpy()[0, 0], (orig_w.item(), orig_h.item()))
            polygons = mask_to_polygons(mask > 0.5)
            preds[name[0]] = polygons_to_string(polygons)

    sample_df['polygons'] = sample_df['filename'].map(preds)
    sample_df.to_csv('submission.csv', index=False)
    print("🎉 모든 과정 완료! submission.csv를 제출하세요.")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)
/tmp/ipython-input-1479398000.py:82: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0)),
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settin

config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

🚀 고해상도 학습 시작...


Epoch 1: 100%|██████████| 818/818 [04:22<00:00,  3.12it/s]


...

📊 Val Loss: 0.0859


Epoch 30: 100%|██████████| 818/818 [02:17<00:00,  5.95it/s]


📊 Val Loss: 0.0857
⭐ 모델 저장
🔍 TTA 기반 추론 시작...


Inference:   0%|          | 0/413 [00:35<?, ?it/s]


TypeError: Got unsupported ScalarType BFloat16

In [5]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm import tqdm

# --- 설정 (경로 확인 필수!) ---
BASE_PATH = './data/datasets'
TEST_IMG_DIR = os.path.join(BASE_PATH, 'images/test')
TEST_JSON = os.path.join(BASE_PATH, 'jsons/test.json')
SAMPLE_SUB = os.path.join(BASE_PATH, 'sample_submission.csv')
MODEL_PATH = 'best_model.pth'  # 학습된 가중치 파일명
OUTPUT_CSV = 'submission.csv'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
RESIZE_TARGET = 1024  # 학습 시 설정과 동일하게 유지
BATCH_SIZE = 8       # A100에서는 더 키워도 됩니다.

# --- 데이터셋 및 유틸리티 ---
class TestDataset(Dataset):
    def __init__(self, img_dir, json_path, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)['images']
        self.image_names = list(self.data.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        image = cv2.imread(os.path.join(self.img_dir, img_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:2]
        if self.transform:
            image = self.transform(image=image)['image']
        return image, img_name, (h, w)

def mask_to_polygons(mask, min_area=30):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) < min_area: continue
        epsilon = 0.005 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if len(approx) >= 4:
            polygons.append(approx.reshape(-1, 2).tolist())
    return polygons

def polygons_to_string(polygons):
    if not polygons: return ""
    return "|".join([" ".join([f"{int(p[0])} {int(p[1])}" for p in poly]) for poly in polygons])

# --- 메인 추론 루프 ---
def run_inference():
    print("🔍 고성능 추론 시작 (TTA 적용)...")

    # 1. 모델 로드 (학습 시 사용한 동일한 설정)
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b3", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()

    # 2. 데이터 준비
    test_transform = A.Compose([A.Resize(RESIZE_TARGET, RESIZE_TARGET), A.Normalize(), ToTensorV2()])
    test_ds = TestDataset(TEST_IMG_DIR, TEST_JSON, transform=test_transform)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    preds = {}

    # 3. 예측
    with torch.no_grad():
        for imgs, names, (orig_h, orig_w) in tqdm(test_loader, desc="Inference"):
            imgs = imgs.to(DEVICE)

            # TTA 및 BFloat16 에러 수정 로직
            with autocast('cuda', dtype=torch.bfloat16):
                p1 = torch.sigmoid(model(imgs))
                p2 = torch.sigmoid(model(torch.flip(imgs, dims=[3])))
                mask_batch = (p1 + torch.flip(p2, dims=[3])) / 2

            # 에러 해결 포인트: .float()를 추가하여 float32로 변환 후 numpy로 넘김
            mask_batch = mask_batch.float().cpu().numpy()

            for i in range(len(names)):
                h, w = orig_h[i].item(), orig_w[i].item()
                mask = cv2.resize(mask_batch[i, 0], (w, h))
                polygons = mask_to_polygons(mask > 0.5)
                preds[names[i]] = polygons_to_string(polygons)

    # 4. 제출 파일 저장
    sample_df = pd.read_csv(SAMPLE_SUB)
    sample_df['polygons'] = sample_df['filename'].map(preds).fillna("")
    sample_df.to_csv(OUTPUT_CSV, index=False)
    print(f"🎉 제출 파일 저장 완료: {OUTPUT_CSV}")

if __name__ == "__main__":
    run_inference()

🔍 고성능 추론 시작 (TTA 적용)...


Inference: 100%|██████████| 52/52 [00:18<00:00,  2.80it/s]

🎉 제출 파일 저장 완료: submission.csv
